# Text classification with BERT

This notebook briefly demonstrates fine-tuning a pretrained BERT model to a text classification task.

You probably want to run this notebook with GPU acceleration, as fine-tuning BERT on CPU can be fairly slow even with a comparatively small dataset.

## Configuration

The following variables configure a few aspects of the data, model, and training process. To adapt this example to a different dataset, you'll probably want to change these to match.

Note in particular that we're limiting the number of examples and the maximum sequence length to make training faster. 

When running on GPU, it's also necessary to make sure that the input length and batch size are not so large as to cause a batch to exceed GPU memory. If you're getting a message like `Resource exhausted: OOM when allocating tensor`, try smaller numbers for these.

In [0]:
# Maximum number of examples to read
MAX_EXAMPLES = 1000 # limited to fasten training 

# Maximum length of input sequence in tokens
INPUT_LENGTH = 250 # limited to fit the data in 

# Number of epochs to train for
EPOCHS = 3

# Optimizer learning rate
LEARNING_RATE = 0.00002 # search for optimal for serious training

# Training batch size
BATCH_SIZE = 8

## Package setup

We'll use [keras-bert](https://github.com/CyberZHG/keras-bert). Make sure the package is installed. (`pip` is the Python [package installer](https://pip.pypa.io/en/stable/))

In [2]:
!pip3 install keras-bert

  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=452c13ed387ede5f46e0283cb96cbaac902a7da1624806009f75929b0c0a7ab8
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for keras-transformer: filename=keras_transformer-0.32.0-cp36-none-any.whl size=13266 sha256=358a56cb0943cb64a0eacfb6c3e192d26017cd557306263ea1535a022467d2da
  Stored in directory: /root/.cache/pip/wheels/62/f0/ce/82fa5d024d5ef8e263f26a50dcee23820efe245680ce9c922a
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=4b84b4fec7f3461b2eb155716fb2d1380716b4e800a297ccc01b4debab5c56b9
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=69bc3ac9ed2b4adec1e27df9f41afbb662e25179ce97eeefb7cdd9c121d59118
  Stor

## Tensorflow configuration

We'll need set an environment variable for keras-bert to use `tensorflow.python.keras`. (This is a technical detail that is not related to the model.)

In [0]:
import os

os.environ['TF_KERAS'] = '1'    # Required to use tensorflow.python.keras with keras-bert

## Download dataset

We'll use the IMDB dataset in JSON format. (This should be familiar to you from previous notebooks in this course.)

In [4]:
# Give -nc (--no-clobber) argument so that the file isn't downloaded multiple times 
!wget -nc https://raw.githubusercontent.com/TurkuNLP/Deep_Learning_in_LangTech_course/master/data/imdb_train.json

--2020-04-21 05:53:25--  https://raw.githubusercontent.com/TurkuNLP/Deep_Learning_in_LangTech_course/master/data/imdb_train.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33944099 (32M) [text/plain]
Saving to: ‘imdb_train.json’

imdb_train.json     100%[===================>]  32.37M   134MB/s    in 0.2s    

2020-04-21 05:53:27 (134 MB/s) - ‘imdb_train.json’ saved [33944099/33944099]



## Load data

We load the IMDB data and shuffle it as usual.

In [5]:
import json
import random


with open('imdb_train.json') as f:
    data = json.load(f)

random.seed(1234)    # This makes the shuffle produce the same order every time
random.shuffle(data)

if len(data) > MAX_EXAMPLES: # truncate data if needed
    print('Note: truncating examples from {} to {}'.format(len(data), MAX_EXAMPLES))
    data = data[:MAX_EXAMPLES]

# Look at the data
print(type(data))
print(type(data[0]))
print(data[0].keys())
print(data[0])

Note: truncating examples from 25000 to 1000
<class 'list'>
<class 'dict'>
dict_keys(['class', 'text'])
{'class': 'pos', 'text': "I saw Insomniac's Nightmare not to long ago for the first time and I have to say, I really found it to be quite good. If you are a fan of Dominic Monaghan you will love it. The hole movie takes place inside his mind -or does it? The acting from everyone else is a little rushed and shaky and some of the scenes could be cut down but it works out in the end. The extras on the DVD are just as great as the film, if not greater for those Dom fans. It has tons of candid moments from the set, outtakes and a great interview with the director. Anyone who has gone through making an independent film will love to watch Tess (the director), Dom and everyone else on the very small close personal set try to bang out this little trippy creepy film. It was pretty enjoyable and I'm glad to have it in my collection."}


Separate the texts and labels into lists of their own.

In [6]:
texts = [example['text'] for example in data]
labels = [example['class'] for example in data]

# Example text and label
print('Text:', texts[0])
print('Label:', labels[0])

Text: I saw Insomniac's Nightmare not to long ago for the first time and I have to say, I really found it to be quite good. If you are a fan of Dominic Monaghan you will love it. The hole movie takes place inside his mind -or does it? The acting from everyone else is a little rushed and shaky and some of the scenes could be cut down but it works out in the end. The extras on the DVD are just as great as the film, if not greater for those Dom fans. It has tons of candid moments from the set, outtakes and a great interview with the director. Anyone who has gone through making an independent film will love to watch Tess (the director), Dom and everyone else on the very small close personal set try to bang out this little trippy creepy film. It was pretty enjoyable and I'm glad to have it in my collection.
Label: pos


## Download pretrained BERT model

As training BERT from scratch generally takes days, we'll here load a pretrained model and fine-tune it for our task. URLs to download pre-trained models made available by Google are found at https://github.com/google-research/bert .

`cased_L-12_H-768_A-12` is a case-sensitive BERT "base" model for English: 12 layers, 768-dimensional hidden state, and 12 "heads" for multi-head attention.

In [7]:
# Give -nc (--no-clobber) argument so that the file isn't downloaded multiple times 
!wget -nc https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip

--2020-04-21 05:55:00--  https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 2404:6800:4008:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 404261442 (386M) [application/zip]
Saving to: ‘cased_L-12_H-768_A-12.zip’

cased_L-12_H-768_A- 100%[===================>] 385.53M  62.8MB/s    in 6.7s    

2020-04-21 05:55:07 (57.8 MB/s) - ‘cased_L-12_H-768_A-12.zip’ saved [404261442/404261442]



Unpack the downloaded file:

In [8]:
# Give -n argument so that existing files aren't overwritten 
!unzip -n cased_L-12_H-768_A-12.zip

Archive:  cased_L-12_H-768_A-12.zip
   creating: cased_L-12_H-768_A-12/
  inflating: cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: cased_L-12_H-768_A-12/vocab.txt  
  inflating: cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: cased_L-12_H-768_A-12/bert_config.json  


There are three key parts to the package contents:

* `vocab.txt`: plain text file listing vocabulary items
* `bert_config.json`: model configuration in JSON format
* `bert_model.ckpt.*`: model checkpoint data with pre-trained weights in [Tensorflow checkpoint format](https://www.tensorflow.org/guide/checkpoint)

Take note of the path to these (you'll need to change this if you pick a different model)

In [0]:
bert_vocab_path = 'cased_L-12_H-768_A-12/vocab.txt'
bert_config_path = 'cased_L-12_H-768_A-12/bert_config.json'
bert_checkpoint_path = 'cased_L-12_H-768_A-12/bert_model.ckpt'    # suffixes not required

Also take note if the model we downloaded was a case-sensitive (cased) or not. (This must match the model.)

(words starting with capital letters are not the same as words with lower case: House != house)

In [0]:
model_is_cased = True 

## Load BERT vocabulary

This is just a plain text file with one vocabulary item per line.

In [12]:
vocab = []
with open(bert_vocab_path) as f:
    for i, line in enumerate(f):
        vocab.append(line.rstrip('\n'))    # rstrip to remove newline characters


# Print a list with every 500th vocabulary item (includes suffixes)
print(vocab[0::500])

['[PAD]', 'щ', '吉', 'told', 'space', 'operations', 'proposed', 'Oxford', 'showing', 'domestic', 'mountains', 'commission', 'voices', 'associate', 'hills', 'Guide', 'relaxed', 'Page', 'Heights', 'singers', 'Interior', 'considers', 'facilitate', 'shouting', '1826', 'constitute', 'alter', 'clip', 'Into', 'Memory', 'ballad', 'Owens', 'Langdon', 'aquatic', 'stereo', 'Cass', 'Shock', '195', '##tec', '##sonic', 'attested', '##rdes', '1840s', '##90', 'Guys', '##rien', 'Munro', 'Ursula', 'mesh', 'diplomacy', 'Newmarket', '##oughs', 'synthesizers', 'Drugs', 'monstrous', '##ynamic', 'troll', '##ٹ']


## Load BERT configuration

The configuration is just a JSON file, so we can read it in with `json.load` from the python `json` library.

We won't actually need to use these configuration details directly (keras-bert takes care of them for us), so this is just here to show what information is contained in the config file.

In [13]:
from pprint import pprint    # pretty-printer for output

with open(bert_config_path) as f:
    config = json.load(f)

# Print configuration contents
pprint(config)

{'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 28996}


## Create BERT tokenizer

To create the tokenizer, we'll need a mapping from vocabulary items to their corresponding integer indices. We do this conventionally using `enumerate`.

In [14]:
# Create mapping from vocabulary items to their indices in the vocabulary
token_dict = { v: i for i, v in enumerate(vocab) }

# Print some random examples of the mapping
pprint(dict(random.choices(list(token_dict.items()), k=10)))

{'##FF': 17515,
 '##bi': 5567,
 '##good': 23684,
 'Colours': 23189,
 'Lockheed': 18904,
 'Merrill': 17247,
 'Wheel': 19754,
 '[unused82]': 82,
 'clock': 4705,
 'corridors': 19507}


We'll use the keras-bert `Tokenizer` for BERT tokenization. The implementation supports

* (Optional) lowercasing: `Hello` → `hello`
* Basic tokenization: `Hello!` → `Hello` `!`, `multi-part` → `multi` `-` `part`
* Wordpiece tokenization: `comprehensively` → `comprehensive` `##ly`
* Adding special tokens: `Sentence`  → `[CLS]` `Sentence` `[SEP]`
* Mapping to integer indices
* Generating segment sequence
* (Optional) padding and truncation to length

In the following example, notice how words not in the dictionary are broken up into subwords (with continuation parts starting with `##`) and how unknown _characters_ are mapped to a special unknown word token `[UNK]`.

In [15]:
from keras_bert import Tokenizer

tokenizer = Tokenizer(token_dict, cased=model_is_cased)

# Let's test that out
for s in ['Hello BERT!', 'Unknown: 你']:
    print('Original string:', s)
    print('Tokenized:', tokenizer.tokenize(s))
    indices, segments = tokenizer.encode(s, max_len=20)    # max_len for padding and truncation
    print('Encoded:', indices)
    print('Segments:', segments)
    print('Decoded:', ' '.join(tokenizer.decode(indices)))
    print()

Original string: Hello BERT!
Tokenized: ['[CLS]', 'Hello', 'B', '##ER', '##T', '!', '[SEP]']
Encoded: [101, 8667, 139, 9637, 1942, 106, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Segments: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Decoded: Hello B ##ER ##T !

Original string: Unknown: 你
Tokenized: ['[CLS]', 'Unknown', ':', '你', '[SEP]']
Encoded: [101, 16285, 131, 100, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Segments: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Decoded: Unknown : [UNK]



## Vectorize data

We'll use the familiar `LabelEncoder` for labels and the keras-bert `Tokenizer` for text data. `Y` is the representation of the labels that will be given to the model for training.

In [16]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()    # Turns class labels into integers
Y = label_encoder.fit_transform(labels)

# Take note of how many unique labels there are in the data
num_labels = len(set(Y))

# Print out some examples
print('Number of unique labels:', num_labels)
print(type(labels), labels[:10])
print(type(Y), Y[:10])

Number of unique labels: 2
<class 'list'> ['pos', 'neg', 'pos', 'pos', 'pos', 'neg', 'neg', 'pos', 'neg', 'pos']
<class 'numpy.ndarray'> [1 0 1 1 1 0 0 1 0 1]


Keep token indices and segment ids in separate lists and store as numpy arrays. `X` here is the final vectorized form of the input we'll be providing to the model for training. 

In [17]:
import numpy as np

token_indices, segment_ids = [], []
for text in texts:
    # tokenizer.encode() returns a sequence of token indices
    # and a sequence of segment IDs. BERT expects both as input,
    # even if the segments IDs are just all zeros (like here).
    tid, sid = tokenizer.encode(text, max_len=INPUT_LENGTH)
    token_indices.append(tid)
    segment_ids.append(sid)

# Format input as list of two numpy arrays
X = [np.array(token_indices), np.array(segment_ids)]


# Print some examples
print('Token indices:')
print(X[0][:2])
print('Decoded:')
for i in X[0][:2]:
    print(tokenizer.decode(list(i)))
print('Segment ids:')
print(X[1][:2])

Token indices:
[[  101   146  1486  1130  7301  1306  5813  1665   112   188 26247  1136
   1106  1263  2403  1111  1103  1148  1159  1105   146  1138  1106  1474
    117   146  1541  1276  1122  1106  1129  2385  1363   119  1409  1128
   1132   170  5442  1104 11288 15706 23136  1128  1209  1567  1122   119
   1109  4569  2523  2274  1282  1656  1117  1713   118  1137  1674  1122
    136  1109  3176  1121  2490  1950  1110   170  1376  6169  1105 16591
   1105  1199  1104  1103  4429  1180  1129  2195  1205  1133  1122  1759
   1149  1107  1103  1322   119  1109  3908  1116  1113  1103  4173  1132
   1198  1112  1632  1112  1103  1273   117  1191  1136  3407  1111  1343
  16727  3899   119  1135  1144  5606  1104  1169  3309  1181  4899  1121
   1103  1383   117  1149 13482  1116  1105   170  1632  3669  1114  1103
   1900   119 15859  1150  1144  2065  1194  1543  1126  2457  1273  1209
   1567  1106  2824 16613   113  1103  1900   114   117 16727  1105  2490
   1950  1113  1103  13

## Load pretrained BERT model

We'll use the keras-bert function `load_trained_model_from_checkpoint` to load the model from the checkpoint we downloaded earlier.

Explanation for a few parameters from keras-bert documentation:

* `training`: If `training`, the whole model will be returned. Otherwise, the MLM [masked language modeling] and NSP [next sentence prediction] parts will be ignored.
* `trainable`: Whether the model is trainable. The default value is the same with `training`.

We don't need the masked language modeling or next sentence prediction parts (these are primarily for pretraining), so we'll use `training=False` but `trainable=True`.

In [0]:
from keras_bert import load_trained_model_from_checkpoint

pretrained_model = load_trained_model_from_checkpoint(
    config_file = bert_config_path,
    checkpoint_file = bert_checkpoint_path,
    training = False,
    trainable = True,
    seq_len = INPUT_LENGTH
)

Let's have a bit of a look at that model

In [19]:
# This is a keras model, so we can figure out what inputs it takes like so:
# we do not care about the segments here: all zero
pretrained_model.inputs

[<tf.Tensor 'Input-Token:0' shape=(None, 250) dtype=float32>,
 <tf.Tensor 'Input-Segment:0' shape=(None, 250) dtype=float32>]

In [20]:
# And similarly for outputs:
pretrained_model.outputs

[<tf.Tensor 'Encoder-12-FeedForward-Norm/Identity:0' shape=(None, 250, 768) dtype=float32>]

So, the model takes `Input-Token` and `Input-Segment` inputs, both of dimension (batch-size, input-length), and produces a single output tensor of dimension (batch-size, input-length, hidden-dim). The input matches our `X`, but we'll need to work on the output a bit as our `Y` is just a label for each input, not a sequence of labels.

In [21]:
pretrained_model.summary() # same transformer layer repeated 12 times: self
# attention, feedforward, drop out, normalization

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 250)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 250)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 250, 768), ( 22268928    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 250, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

This is a regular Keras model. In Keras, models behave very much like layers, so we're able to wrap this in our own model.

## Build classification model

We'll make a *very* simple model for text classification: just attach a dense layer to the output for the special `[CLS]` token, and connect the model inputs to the BERT model inputs.

Recall that in the BERT input representation, each sequence starts with the special `[CLS]` token and the corresponding output is used in BERT pretraining for the next sentence prediction task. We'll use the `[CLS]` output similarly for our classification task.

<img src="https://raw.githubusercontent.com/TurkuNLP/Deep_Learning_in_LangTech_course/master/bert-representation.png" style="width: 80%">

(Figure from [Devlin et al.](https://arxiv.org/pdf/1810.04805.pdf))

First, let's find the output

In [22]:
print(pretrained_model.outputs)

[<tf.Tensor 'Encoder-12-FeedForward-Norm/Identity:0' shape=(None, 250, 768) dtype=float32>]


Those dimensions are (minibatch-size, sequence-length, hidden-dim).

We'll just need the first sequence position across all elements in the initial (minibatch) dimension:

In [23]:
# model.outputs is a list, here with a single item. Here
# pretrained_model.outputs[0] just grabs that item (the output tensor).
# Indxing that tensor with [:,0] gives the first position in the sequence
# for all elements in the batch (the `:`).
bert_out = pretrained_model.outputs[0][:,0]

print(bert_out)

Tensor("strided_slice:0", shape=(None, 768), dtype=float32)


Then we can simply create our model. This is just basic Keras, where the pretrained BERT model is behaving essentially as a layer of our "wrapping" model.

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense

out = Dense(num_labels, activation='softmax')(bert_out) # pretrained model output fed to a simple model!
model = Model(
    inputs=pretrained_model.inputs,
    outputs=[out]
)

## Create optimizer

BERT is pretrained with an Adam optimizer with warmup and regularization using weight decay. We won't go into detail on these optimizer settings, but will instead largely copy parameters used in the original BERT work.

(If you're interested in tuning the training process, trying different values of `LEARNING_RATE` is a good place to start!)

In [0]:
from keras_bert import calc_train_steps, AdamWarmup


# Calculate the number of steps for warmup
total_steps, warmup_steps = calc_train_steps(
    num_example=len(texts),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    warmup_proportion=0.1,
)

optimizer = AdamWarmup(
    total_steps,
    warmup_steps,
    lr=LEARNING_RATE,
    epsilon=1e-6,
    weight_decay=0.01,
    weight_decay_pattern=['embeddings', 'kernel', 'W1', 'W2', 'Wk', 'Wq', 'Wv', 'Wo']
)

## Train model

The model is compiled and trained normally. As usual, we'll use `sparse_categorical_crossentropy` loss.

In [0]:
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

Training, as usual. (Note: this will take a fair bit of time unless you're running with GPU acceleration.)

In [0]:
history = model.fit(
    X,
    Y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.1
)

Epoch 1/3
 36/113 [========>.....................] - ETA: 29:55 - loss: 0.6966 - sparse_categorical_accuracy: 0.5590

Let's plot that training history:

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt


def plot_history(history):
    plt.plot(history.history['sparse_categorical_accuracy'],label="Training set accuracy")
    plt.plot(history.history['val_sparse_categorical_accuracy'],label="Validation set accuracy")
    plt.legend()
    plt.show()


plot_history(history)

This is definitely starting to overfit by the third epoch, but the best dev set result is pretty impressive for such a small subset of the training data.

(Caveat: by downsampling all of the data, we also ended up with a small dev set, so these results may have substantial variance.)

## Predict with the trained model

Let's run a few simple cases through the trained model to illustrate its use.

In [0]:
def predict_one(text):
    # This function takes a string, vectorizes it with the tokenizer,
    # and returns the model prediction. The vectorization follows
    # the pattern we saw above for training data preparation:
    tid, sid = tokenizer.encode(text, max_len=INPUT_LENGTH)
    test_token_indices = [tid]
    test_segment_ids = [sid]
    test_X = [np.array(test_token_indices), np.array(test_segment_ids)]
    # Run model.predict for this single case 
    predictions = model.predict(test_X)
    # model.predict always returns a sequence. As we there's only a single input,
    # we'll just return the value of the first (and only) element.
    return predictions[0]


# Get the index of the positive label ("pos"). Note that as LabelEncoder works
# on lists, we need to wrap the label in a list when calling it, and take the
# first (and only) element from the returned sequence.
pos_index = label_encoder.transform(['pos'])[0]


# Test the model with a few strings of our own.
test_strings = [
  'This movie is amazing!',
  'This movie is amazingly bad!',
  'The plot is boring, but the action is exciting.',
  'The plot is exciting, but the action is boring.',
  'I am not sure what to think about this movie.',
]


for s in test_strings:
    pred = predict_one(s)
    # The model returns the probability of both labels (pos/neg) where the
    # neg = 1-pos (softmax). Just print out the positive label probability.
    print('Positive probability {:.1%}\t{}'.format(pred[pos_index], s))